In [1]:
import open3d as o3d
import numpy as np

class BulldogGenerator:
    def __init__(self):
        self.components = []
        self.defaults = {
            'body_scale': [1.5, 1.0, 0.6],
            'head_scale': [0.7, 0.7, 0.5],
            'leg_radius': 0.15,
            'leg_height': 0.4
        }

    def _create_part(self, shape, params, position):
        mesh = getattr(o3d.geometry.TriangleMesh, f'create_{shape}')(**params)
        mesh.compute_vertex_normals()
        mesh.translate(position)
        return mesh

    def _add_body(self, features):
        body = self._create_part('sphere', {'radius': 1}, (0,0,0))
        body.scale(self.defaults['body_scale'], center=(0,0,0))
        self.components.append(body)
        
        # Head
        head_pos = (self.defaults['body_scale'][0]/2 + 0.3, 0, 0)
        head = self._create_part('sphere', {'radius': 0.7}, head_pos)
        head.scale(self.defaults['head_scale'], center=head_pos)
        self.components.append(head)
        
        if 'wrinkled' in features:
            vertices = np.asarray(head.vertices)
            head.vertices = o3d.utility.Vector3dVector(vertices + np.random.normal(0, 0.02, vertices.shape))

    def _add_legs(self):
        positions = [
            (0.4, 0.4, -self.defaults['body_scale'][2]),
            (-0.4, 0.4, -self.defaults['body_scale'][2]),
            (0.4, -0.4, -self.defaults['body_scale'][2]),
            (-0.4, -0.4, -self.defaults['body_scale'][2])
        ]
        for pos in positions:
            leg = self._create_part('cylinder', 
                {'radius': self.defaults['leg_radius'], 'height': self.defaults['leg_height']}, 
                pos
            )
            self.components.append(leg)

    def _add_face(self):
        # Simple eyes
        for x in [0.2, -0.2]:
            eye = self._create_part('sphere', {'radius': 0.08}, 
                (self.defaults['body_scale'][0]/2 + 0.3 + x, 0.15, 0.1))
            self.components.append(eye)

    def generate(self, description=""):
        features = [f for f in ['wrinkled', 'short', 'collar'] if f in description.lower()]
        
        self._add_body(features)
        self._add_legs()
        self._add_face()
        
        # Combine all parts
        full_mesh = self.components[0]
        for part in self.components[1:]:
            full_mesh += part
            
        return full_mesh

# Usage
generator = BulldogGenerator()
bulldog = generator.generate("wrinkled short bulldog")
o3d.visualization.draw_geometries([bulldog])

TypeError: scale(): incompatible function arguments. The following argument types are supported:
    1. (self: open3d.cpu.pybind.geometry.Geometry3D, scale: float, center: numpy.ndarray[numpy.float64[3, 1]]) -> open3d.cpu.pybind.geometry.Geometry3D
    2. (self: open3d.cpu.pybind.geometry.Geometry3D, scale: float, center: numpy.ndarray[numpy.float64[3, 1]]) -> open3d.cpu.pybind.geometry.Geometry3D

Invoked with: TriangleMesh with 762 points and 1520 triangles., [1.5, 1.0, 0.6]; kwargs: center=(0, 0, 0)

In [10]:
# Cell 1: Core Components
import open3d as o3d
import numpy as np

DEFAULT_PARAMS = {
    'body_scale': [1.5, 1.0, 0.6],
    'head_scale': [0.7, 0.7, 0.5],
    'leg_radius': 0.15,
    'leg_height': 0.4
}

def create_part(shape, params, position=(0,0,0)):
    """Base shape creation function with extension hooks"""
    # Extension point: Add new shape types here
    mesh = getattr(o3d.geometry.TriangleMesh, f'create_{shape}')(**params)
    mesh.compute_vertex_normals()
    mesh.translate(position)
    return mesh

def apply_wrinkles(mesh, intensity=0.02):
    """Modifier function with extension potential"""
    vertices = np.asarray(mesh.vertices)
    mesh.vertices = o3d.utility.Vector3dVector(vertices + np.random.normal(0, intensity, vertices.shape))
    return mesh

In [ ]:
# Enhanced Cell 2: Reasoning System
class FeatureAnalyzer:
    def __init__(self):
        self.reasoning_log = []
        self.detected_features = []
        self.anatomy_map = {
            'body': {'required': True, 'built': False},
            'head': {'required': True, 'built': False},
            'legs': {'required': True, 'built': False},
            'ears': {'requi red': False, 'built': False},
            'tail': {'required': False, 'built': False},
            'collar': {'required': False, 'built': False}
        }
    
    def log_step(self, message):
        self.reasoning_log.append(f"• {message}")
    
    def analyze_description(self, description):
        text = description.lower()
        self.detected_features = []
        
        # Enhanced feature detection
        feature_triggers = {
            'wrinkled': ['wrinkle', 'wrinkled', 'fold'],
            'short_legs': ['short legs', 'stubby legs'],
            'floppy_ears': ['floppy ears', 'droopy ears'],
            'stubby_tail': ['stubby tail', 'short tail'],
            'collar': ['collar', 'neckband', 'leather']
        }
        
        for feature, keywords in feature_triggers.items():
            if any(kw in text for kw in keywords):
                self.detected_features.append(feature)
                self.log_step(f"Detected {feature.replace('_', ' ')} feature")
        
        # Anatomy validation
        for part in self.anatomy_map:
            if self.anatomy_map[part]['required'] and not self.anatomy_map[part]['built']:
                self.log_step(f"Warning: Missing required {part} component")
        
        return self.detected_features

In [12]:
# Enhanced Cell 3: Generator Class & Usage
class BulldogGenerator:
    def __init__(self):
        self.components = []
        self.analyzer = FeatureAnalyzer()
        self.transform_stack = []
    
    def _apply_scaling(self, mesh, scale_factors):
        scaling_matrix = np.eye(4)
        scaling_matrix[0, 0] = scale_factors[0]
        scaling_matrix[1, 1] = scale_factors[1]
        scaling_matrix[2, 2] = scale_factors[2]
        mesh.transform(scaling_matrix)
        return mesh
    
    def _get_body_dimensions(self):
        return (
            DEFAULT_PARAMS['body_scale'][0],
            DEFAULT_PARAMS['body_scale'][1],
            DEFAULT_PARAMS['body_scale'][2]
        )
    
    def _add_body(self, features):
        # Main body
        body = create_part('sphere', {'radius': 1})
        self._apply_scaling(body, DEFAULT_PARAMS['body_scale'])
        self.components.append(body)
        self.analyzer.log_step("Created scaled body")
        
        if 'wrinkled' in features:
            apply_wrinkles(body, intensity=0.01, area='body')
            self.analyzer.log_step("Added body wrinkles")

        # Head with proper alignment
        body_width = self._get_body_dimensions()[0]
        head_pos = (body_width/2 + 0.3, 0, 0.2)  # Offset for neck area
        head = create_part('sphere', {'radius': 0.7}, head_pos)
        self._apply_scaling(head, DEFAULT_PARAMS['head_scale'])
        
        if 'wrinkled' in features:
            apply_wrinkles(head, intensity=0.03)
            self.analyzer.log_step("Added detailed facial wrinkles")
            
        self.components.append(head)
        self.analyzer.anatomy_map['head']['built'] = True

        # Neck connection
        neck = create_part('cylinder', 
                          {'radius': 0.2, 'height': 0.3},
                          (body_width/2 + 0.15, 0, 0.1),
                          rotation=[0, 20, 0])
        self.components.append(neck)
    
    def _add_legs(self):
        body_depth = self._get_body_dimensions()[2]
        positions = [
            (0.4, 0.4, -body_depth),
            (-0.4, 0.4, -body_depth),
            (0.4, -0.4, -body_depth),
            (-0.4, -0.4, -body_depth)
        ]
        
        for pos in positions:
            leg = create_part('cylinder', {
                'radius': DEFAULT_PARAMS['leg_radius'],
                'height': DEFAULT_PARAMS['leg_height'],
                'resolution': 16
            }, pos)
            
            # Add paw details
            paw = create_part('sphere', 
                            {'radius': DEFAULT_PARAMS['leg_radius']*1.2},
                            (pos[0], pos[1], pos[2] - DEFAULT_PARAMS['leg_height']/2))
            self.components.append(paw)
            
            self.components.append(leg)
        
        self.analyzer.log_step("Added detailed legs with paws")
        self.analyzer.anatomy_map['legs']['built'] = True
    
    def _add_face(self, features):
        # Eyes
        body_width = self._get_body_dimensions()[0]
        for x in [0.2, -0.2]:
            eye = create_part('sphere', {'radius': 0.08}, 
                            (body_width/2 + 0.3 + x, 0.15, 0.3))
            self.components.append(eye)
        
        # Nose
        nose = create_part('cylinder', 
                         {'radius': 0.1, 'height': 0.05},
                         (body_width/2 + 0.3, 0, 0.2),
                         rotation=[90, 0, 0])
        self.components.append(nose)
        
        # Mouth
        mouth_points = [
            (body_width/2 + 0.3 + 0.1*n, 0.05, 0.15) 
            for n in range(-3, 4)
        ]
        for point in mouth_points:
            mouth = create_part('sphere', {'radius': 0.02}, point)
            self.components.append(mouth)
        
        self.analyzer.log_step("Added detailed face with eyes, nose, and mouth")
    
    def _add_ears(self, features):
        if 'floppy_ears' in features:
            # Floppy ears
            ear_positions = [
                (self._get_body_dimensions()[0]/2 + 0.3, 0.25, 0.4),
                (self._get_body_dimensions()[0]/2 + 0.3, -0.25, 0.4)
            ]
            for pos in ear_positions:
                ear = create_part('cylinder', 
                                {'radius': 0.08, 'height': 0.3},
                                pos,
                                rotation=[-45, 0, 45 if pos[1] > 0 else -45])
                self.components.append(ear)
            self.analyzer.log_step("Added floppy ears")
    
    def _add_tail(self, features):
        if 'stubby_tail' in features:
            tail_pos = (-self._get_body_dimensions()[0]/2 - 0.1, 0, 0.1)
            tail = create_part('cone', 
                             {'radius': DEFAULT_PARAMS['tail_radius'], 
                              'height': 0.2},
                             tail_pos,
                             rotation=[0, -45, 0])
            self.components.append(tail)
            self.analyzer.log_step("Added stubby tail")
    
    def _add_collar(self, features):
        if 'collar' in features:
            collar_pos = (self._get_body_dimensions()[0]/2 - 0.2, 0, 0.3)
            collar = create_part('torus', 
                               {'radius': DEFAULT_PARAMS['collar_radius'],
                                'tube_radius': 0.03,
                                'radial_resolution': 32,
                                'tubular_resolution': 16},
                               collar_pos,
                               rotation=[0, 90, 0])
            self.components.append(collar)
            self.analyzer.log_step("Added detailed leather collar")
    
    def generate(self, description):
        self.components = []
        features = self.analyzer.analyze_description(description)
        
        # Build order matters for proper alignment
        self._add_body(features)
        self._add_legs()
        self._add_face(features)
        self._add_ears(features)
        self._add_tail(features)
        self._add_collar(features)
        
        # Merge components with transform preservation
        full_mesh = self.components[0]
        for part in self.components[1:]:
            full_mesh += part
            
        return full_mesh

# Usage example
generator = BulldogGenerator()
bulldog = generator.generate(
    "A wrinkled bulldog with floppy ears, stubby tail, and leather collar"
)
print("\n".join(generator.analyzer.reasoning_log))
o3d.visualization.draw_geometries([bulldog],
                                 window_name="Enhanced 3D Bulldog",
                                 width=1200,
                                 height=800)

TypeError: apply_wrinkles() got an unexpected keyword argument 'area'